## Master Agreement/PSA Report 
* Goal: Make it easy for branch chiefs to know whether there is a Master Agreement and/or Program Supplement is needed or going to expire in the next year through a monthly report.


In [1]:
import A1_data_prep
import A2_tableau
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Project Sheet
•	Column A - Award Year  
•	Column B - Project #       
•	Column C - Grant Recipient          
•	Column D - Project Title 
•	Column E - PPNO             
•	Column I - Master Agreement Number
•	Column J - Master Agreement Expiration Date
•	Column K - Project Manager

In [2]:
# Load in sheets
project = A1_data_prep.clean_project()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# Subset of cols
project_cols = [
    "project_award_year",
    "project_project_#",
    "project_grant_recipient",
    "project_project_title",
    "project_ppno",
    "project_master_agreement_number",
    "project_master_agreement_expiration_date",
    "project_project_manager",
]

In [4]:
# Subset df to only the columns requested
project2 = project[project_cols]

In [21]:
project2.project_master_agreement_expiration_date = project2.project_master_agreement_expiration_date.apply(pd.to_datetime, errors="coerce")

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


* If the master agreement field (column I, tab 1) is empty, the information should appear on the report
* If there is less than a year until the Master Agreement Expiration Date, the information should appear on the report

In [54]:
# Filter based on criteria above
project3 = (project2[
    ((project2.project_master_agreement_expiration_date.dt.year == 2023) | (project2.project_master_agreement_number.isin(["None","Pending"])))
]).reset_index(drop = True)

In [55]:
project3

,project_award_year,project_project_#,project_grant_recipient,project_project_title,project_ppno,project_master_agreement_number,project_master_agreement_expiration_date,project_project_manager
0,2016,14,Santa Clara Valley Transportation Authority,BART Silicon Valley Phase II Extension,CP057,None,NaT,Leela Immadisetty
1,2018,21,Santa Barbara County Association Of Governments,Coastal Express/Pacific Surfliner Peak Hour Service Expansion and Integration Project,CP048,64SBCAGMA,2023-10-01,Dina Facchini
2,2018,22,Santa Barbara County Association Of Governments,Goleta Train Depot,CP047,64SBCAGMA,2023-10-01,Dina Facchini
3,2020,5,Lake Transit Authority,North State Intercity Bus System,CP063,64LTAMA,2023-06-30,Gary Atwal
4,2022,5,City Of Cupertino,South Bay Microtransit Expansion,CP105,Pending,NaT,Deanna Mcnurlan
5,2022,6,City Of Glendale,Making a Beeline for Electrification - City of Glendale and Arroyo Verdugo Communities Zoom towards Cleaner Transportation,CP082,Pending,NaT,Marissa Brown
6,2022,7,City Of Oakland,Oakland Waterfront Mobility Hub,CP086,Pending,NaT,Doug Adams
7,2022,8,City Of Torrance,The Regional Connectivity Improvement Bus Program,CP095,Pending,NaT,Yesenia Ochoa
8,2022,9,City Of Wasco,City of Wasco Improving Air Quality and Economic Growth with Bus Electrification,CP090,Pending,NaT,Nedina Facchini
9,2022,10,Contra Costa Transportation Authority,I-680 Express Bus Program,CP096,Pending,NaT,Leela Immadisetty


### Allocation Sheet
•	Column F - Project ID
•	Column E - EA
•	Column I - Phase 
•	Column J - Allocation Amount
•	Column T - Allocation Date 
•	Column V - PSA #
•	Column AB - Date Branch Chief Receives PSA       
•	Column AC - Date Regional Coordinator Receives PSA      
•	Column AD - Date OC Receives PSA          
•	Column AE - Date OPM Receives PSA      
•	Column AF - Date Legal Receives PSA      
•	Column AG - Date Returned to PM           
•	Column AH - Date PSA Sent to Local Agency         
•	Column AI - Date PSA Approved by Local Agency 
•	Column AJ - Date Signed by DRMT
•	Column AK – PSA Expiry Date


In [5]:
alloc = A1_data_prep.clean_allocation()

In [6]:
# Subset of cols
alloc_cols = [
    "allocation_ppno",
    "allocation_project_id",
    "allocation_ea",
    "allocation_phase",
    "allocation_allocation_amount",
    "allocation_allocation_date",
    "allocation_psa_#",
    "allocation_date_branch_chief_receives_psa",
    "allocation_date_regional_coordinator_receives_psa",
    "allocation_date_oc_receives_psa",
    "allocation_date_opm_receives_psa",
    "allocation_date_legal_receives_psa",
    "allocation_date_returned_to_pm",
    "allocation_date_psa_sent_to_local_agency",
    "allocation_date_psa_approved_by_local_agency",
    "allocation_date_signed_by_drmt",
    "allocation_psa_expiry_date",
]

In [8]:
alloc2 = alloc[alloc_cols]

* If there is a date in the “Allocation Date” field (column T) and no date in the in “Date Signed by DRMT” (column AJ), then the information should be on the report
* If there is 6 months or less until the expiry date (column AK), then the information would be on the report


In [49]:
alloc3 = (alloc2[(
    (alloc2.allocation_allocation_date.notna()) &
         (alloc2.allocation_date_signed_by_drmt.isna())) |
        (alloc2.allocation_psa_expiry_date.dt.date.astype(str) < '2023-06-12')
    ]).reset_index()

In [56]:
alloc3

,allocation_ppno,allocation_project_id,allocation_ea,allocation_phase,allocation_allocation_amount,allocation_allocation_date,allocation_psa_#,allocation_date_branch_chief_receives_psa,allocation_date_regional_coordinator_receives_psa,allocation_date_oc_receives_psa,allocation_date_opm_receives_psa,allocation_date_legal_receives_psa,allocation_date_returned_to_pm,allocation_date_psa_sent_to_local_agency,allocation_date_psa_approved_by_local_agency,allocation_date_signed_by_drmt,allocation_psa_expiry_date
1,CP012,None,R350GA,CONST,4620000.00,2016-05-19,VARCCJPAPS-01 A1,NaT,NaT,NaT,NaT,NaT,NaT,None,NaT,2016-12-13,2019-06-01
2,CP015,None,R353GA,CONST,38494000.00,2016-06-30,\n07LACMTAPS-01 A1\n,NaT,NaT,NaT,NaT,NaT,NaT,None,NaT,2017-08-04,2020-06-28
3,CP007,0016000119,R345GA,CONST,1675000.00,2015-12-10,VARLOSSANPS-01 A3\n,NaT,NaT,NaT,NaT,NaT,NaT,None,2020-01-20,2020-01-31,2021-06-30
4,CP013,None,T349GA,CONST,10000000.00,2016-05-19,05MST2015PS-01,NaT,NaT,NaT,NaT,NaT,NaT,None,2016-11-11,2016-11-21,2019-11-03
11,CP006,None,R344GA,CONST,41181000.00,2015-12-10,04SFMTA2015PS-01,NaT,NaT,NaT,NaT,NaT,NaT,None,2017-09-26,2017-10-11,2018-12-10
12,CP014,None,R351GA,PS&E,15000.00,2017-08-17,10SJRRCPS-01,NaT,NaT,NaT,NaT,NaT,NaT,None,2018-01-12,2018-02-08,2020-06-30
13,CP014,None,R351GB,CONST,185000.00,2018-02-01,10SJRRCPS-01 A1,NaT,NaT,NaT,NaT,NaT,NaT,None,2018-06-25,2018-07-12,2021-02-02
14,CP011,None,T348GA,CONST,6841000.00,2016-03-16,10SJRTDPS-01 A1,NaT,NaT,NaT,NaT,NaT,NaT,None,2017-09-20,2017-09-28,2020-06-23
15,CP000,None,R341GA,CONST,41181000.00,2015-08-27,07SCRRAPS-01 A1\nPending,NaT,NaT,NaT,2022-07-26,2022-08-01,2022-08-22,None,2019-12-18,2022-10-03,2021-09-30
26,CP016,None,T357GB,CONST,3917000.00,2020-06-25,06FRESNOPS-01 A4,NaT,NaT,NaT,2022-03-11,2022-03-16,2022-03-16,4/5/2022\nResent 7/27/22,NaT,NaT,NaT


* If there is new information in column Q and no change in column, then the information should be on the report